## Reed-Solomon/Reed-Solomon Product Code

In [32]:
class RSRSCode:
    
    def __init__(self, n, k, q, alpha = None):
        
        if not (k < n and n <= q):
            raise ValueError('Invalid values for n, k, and q.')
            
        self.p0, self.m = is_prime_power(q, get_data = True)
            
        self.n = n
        self.k = k
        self.q = q
        
        self.d = self.n - self.k + 1
        
        self.tau = floor((self.n-self.k)/2)
        
        # Initializing field
        self.F = GF(self.q)
        self.R = PolynomialRing(self.F, 'X')
        self.p = self.F.primitive_element()
        
        # Constructing alpha-vector
        if not alpha:
            self.alpha = vector([self.p**i for i in range(self.n)])
        else:
            self.alpha = alpha
        
        # Constructing generator matrix
        self.G = matrix(self.F, k, n, lambda i,j : self.alpha[j]**i)
        
    def Encoding(self, m, zeropad = True):
        
        m = self._IntToPol(m)
        
        rem = len(m) % (self.k * self.k)
        
        if rem != 0:
            if zeropad:
                m.extend([self.F(0)]*(self.k*self.k-rem))
            else:
                raise ValueError('k does not divide input size')
                
                
        c_rows = []
        
        # Encoding each row of size k
        for i in range(0, len(m), self.k):
            c_rows.extend(self.EncodeChunk(m[i:i+self.k]))
        

        c_rows_cols = []
        
        for j in range(self.n): # hence only works for one block
            current_word = []
            for i in range(self.k):
                current_word.append(c_rows[j+i*self.n])
            c_rows_cols.extend(self.EncodeChunk(current_word))
            
        
        #return m
        return c_rows_cols
            
    def EncodeChunk(self, chunk):
        
        # Encode a chunk of size k
        if len(chunk) != self.k:
            raise ValueError('Invalid chunk size')
            
        c = vector(self.F, chunk) * self.G
        return c
    
    def Decoding(self, r):
        
        # Check input size
        if len(r) % (self.n * self.n) != 0:
            raise ValueError('Invalid input size')
            
        c_rows = []
        
        for i in range(0,len(r),self.n):
            c_rows.extend(self.BivariateInterpolation(r[i:i+self.n]))
            
        
        c = []
        
        for j in range(self.k):
            current_word = []
            for i in range(self.n):
                #current_word.extend([1])
                current_word.append(c_rows[j+i*self.k])
            c.extend(self.BivariateInterpolation(current_word))
        
        c = self._PolToInt(c)
        return c
    
    def BivariateInterpolation(self, chunk):
        
        if len(chunk) != self.n:
            raise ValueError('Invalid chunk size')
            
        # Constructing matrices
        M1 = matrix(self.F, self.n, self.tau + self.k, lambda i,j : self.alpha[i]**j)
        M2 = matrix(self.F, self.n, self.tau + 1, lambda i,j : chunk[i] * self.alpha[i]**j)
        M = M1.augment(M2)
        
        # Solving system
        RK = M.right_kernel()
        
        if len(RK.basis()) == 0:
            return(None)
        
        sol = RK.basis()[0]

        # Constructing Q0 and Q1 polynomials
        Q0 = self.R(list(sol[:self.tau+self.k]))
        Q1 = self.R(list(sol[self.tau+self.k:]))

        # Calculating -Q0/Q1
        q, r = Q0.quo_rem(Q1)

        if r != 0:
            #print('Non-zero remainder (possibly >tau errors). Returning None')
            return(None)

        out = []

        out.extend((-q).list())
        out.extend([self.F(0)]*(self.k-len(out)))

        return out
    
    def _IntToPol(self, m):
        # Convert array of integers less than q to elements of field
        
        m_out = []
        
        for i in m:
            if not i < self.q:
                raise ValueError('Invalid symbol')
            m_out.append(self.F(ZZ(i).digits(self.p0)))
            
        return m_out
    
    def _PolToInt(self, pol_array):

        # Converts array of integers less than q to elements of Field.

        pol_out = []

        for pol in pol_array:
            if not pol in self.F:
                raise ValueError('Invalid symbol')

            pol_out.append(ZZ(pol.polynomial().coefficients(sparse = False), base = self.p0))

        return(pol_out)

    def _IntToByteString(self, int_array):

        if(self.q != 2^4):
            raise ValueError('Invalid field size for byte representation')

        # Converts array of integers less than 256 to binary representation

        if any([(item > 15 or item < 0) for item in int_array]):
            raise ValueError('Invalid integer values')

        return list(''.join([format(item, '04b') for item in int_array]))

In [33]:
C = RSRSCode(15,7,2**4)

In [34]:
m = [1, 2, 3, 4, 5, 6, 7,
     2, 3, 4, 5, 6, 7, 1,
     3, 4, 5, 6, 7, 1, 2,
     4, 5, 6, 7, 1, 2, 3,
     5, 6, 7, 1, 2, 3, 4,
     6, 7, 1, 2, 3, 4, 5,
     7, 1, 2, 3, 4, 5, 6]
     

c = C.Encoding(m)
print(c)
d = C.Decoding(c)
print(d)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, z4^2 + z4 + 1, z4^3, z4^2 + z4, z4^2 + z4 + 1, z4^3 + z4 + 1, z4^3 + z4^2, z4^3 + 1, z4^3 + z4^2 + z4, z4^3 + z4 + 1, z4^3 + 1, 0, 0, z4^3, 0, z4^2 + z4 + 1, z4^3 + z4 + 1, z4^3 + z4^2, z4^2 + z4 + 1, z4^2 + z4 + 1, z4 + 1, z4 + 1, 0, z4^2 + 1, z4^3 + z4^2 + 1, 0, z4^3 + z4^2, z4^3 + z4^2 + z4, z4^3 + z4 + 1, 0, z4^3, z4^3 + z4^2, 0, 0, z4, z4^3 + 1, z4^2 + z4 + 1, z4^2 + 1, z4^2 + z4, z4^2 + 1, z4^3 + z4^2 + z4 + 1, 1, z4^3 + 1, z4^3 + z4^2 + z4 + 1, 0, z4^2 + z4, z4^2 + z4 + 1, 0, z4^2 + 1, z4^3 + z4^2 + z4 + 1, z4^3 + z4^2 + z4 + 1, z4 + 1, z4^3 + z4, 1, z4^3 + z4^2 + z4, z4^3 + z4^2 + z4 + 1, z4^3 + z4^2 + z4 + 1, z4 + 1, z4^3 + z4^2 + z4, 0, z4^2 + z4 + 1, z4^2 + z4 + 1, z4, z4^3 + z4^2 + z4 + 1, 1, 1, z4^3 + 1, z4, z4^2, z4^2 + z4 + 1, z4^3 + z4^2 + 1, z4^3 + z4^2 + z4, 1, z4^3 + z4^2, 0, z4^3 + z4 + 1, z4 + 1, z4^3 + 1, z4^3 + z4^2 + z4 + 1, 1, z4^2 + z4, z4^3 + z4^2 + z4, z4^2, z4^2 + z4, z4^3 + z4^2, z4^3 + z4^2 + z4, z4^3 + 